In [2]:
from ximea import xiapi
import cv2
import time
import os
from PIL import Image
import numpy as np
### runn this command first echo 0|sudo tee /sys/module/usbcore/parameters/usbfs_memory_mb  ###
 
#create instance for first connected camera
cam = xiapi.Camera()

TRACKBAR

In [3]:
def process_image():
    global param1, param2, min_radius, max_radius
    gray_blur = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                               param1=param1, param2=param2,
                               minRadius=min_radius, maxRadius=max_radius)

    output = img.copy()
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(output, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(output, (i[0], i[1]), 2, (0, 0, 255), 3)

    cv2.imshow('HoughCircles', output)

# Trackbar callback functions to update parameters
def update_param1(val):
    global param1
    param1 = val
    process_image()

def update_param2(val):
    global param2
    param2 = val
    process_image()

def update_min_radius(val):
    global min_radius
    min_radius = val
    process_image()

def update_max_radius(val):
    global max_radius
    max_radius = val
    process_image()

# Read the input image
img = cv2.imread('saved_imgs/img0.jpg', cv2.IMREAD_COLOR)
img = cv2.resize(img, (720, 720))
if img is None:
    raise Exception("Could not read the image. Make sure the path is correct.")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Initial HoughCircles parameters
param1 = 100
param2 = 30
min_radius = 10
max_radius = 50

# Create a named window
cv2.namedWindow('HoughCircles')

# Create trackbars
cv2.createTrackbar('Param1', 'HoughCircles', param1, 300, update_param1)
cv2.createTrackbar('Param2', 'HoughCircles', param2, 100, update_param2)
cv2.createTrackbar('Min Radius', 'HoughCircles', min_radius, 100, update_min_radius)
cv2.createTrackbar('Max Radius', 'HoughCircles', max_radius, 200, update_max_radius)

# Initial processing
process_image()

# Wait until user exits
cv2.waitKey(0)
cv2.destroyAllWindows()


IMAGE LOADING

In [2]:

def find_circles(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.medianBlur(gray_img, 5)

    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,dp=1,minDist=50,param1=125,param2=40,minRadius=0,maxRadius=0)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        return circles
    
    else:
        return None





LIVE CAMERA FRAMES

In [5]:
#start communication
#to open specific device, use:
#cam.open_device_by_SN('41305651')
#(open by serial number)
print('Opening first camera...')
cam.open_device()
 
#settings
cam.set_exposure(10000)
cam.set_param('imgdataformat','XI_RGB32')
cam.set_param('auto_wb', 1)
print('Exposure was set to %i us' %cam.get_exposure())
 
#create instance of Image to store image data and metadata
img = xiapi.Image()
 
#start data acquisition
print('Starting data acquisition...')
cam.start_acquisition()
 
i = 0

 
while cv2.waitKey(1) != ord('q'):
    cam.get_image(img)
    image = img.get_image_data_numpy()
    image = cv2.resize(image,(1024,1024))

    circles = find_circles(image)


    if circles is not None:
        for (x, y, r) in circles:
            cv2.circle(image, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(image, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

    

    if cv2.waitKey(1) == ord('s'):
        cv2.imwrite(f"saved_imgs/img{i}.jpg", image)
        i += 1

    cv2.imshow("camera", image)

 
 
#stop data acquisition
print('Stopping acquisition...')
cam.stop_acquisition()
 
#stop communication
cam.close_device()
cv2.destroyAllWindows()
print('Done.')

Opening first camera...
Exposure was set to 10001 us
Starting data acquisition...
Stopping acquisition...
Done.
